# Laboratorio 6. Análisis de Sentimientos
- José Mérida
- Joaquín Puente

## Imports / Librerías Utilizadas

In [21]:
import tensorflow as tf
import numpy as np

# Model building
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Embedding, LSTM, Dense, Dropout,
                                   Bidirectional, concatenate, BatchNormalization)

# Training components
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Data and preprocessing
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# Sentiment analysis for feature extraction
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download("vader_lexicon", quiet=True)  # Set to True to reduce output

True

## 1. Importación de Datos
Importamos los datos de IMDB, utilizando las 50,000 palabras más frecuentes.

In [22]:
print('Cargando los datos...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=50000)
print('Datos Cargados')
print(f"Length x_train: {len(x_train)}, Length y_train: {len(y_train)}")

Cargando los datos...
Datos Cargados
Length x_train: 25000, Length y_train: 25000


Verificamos correctamente que el conjunto tiene 25,000 datos de entrenamiento y 25,000 datos de prueba. Los datos han sido cargados exitosamente.

#2. Pre Procesamiento
Para esta sección, implementamos 3 features adicionales al set de datos. Utilizando SentimentIntensityAnalyzer de nltk, analizamos palabra por palabra del review para obtener un ratio de palabras positivas y negativas dentro del texto. Adicionalmente, agregamos como atributo la longitud original del mensaje.

In [23]:

# Inicializar analizador de sentimiento
analyzer = SentimentIntensityAnalyzer()

# Mapeo de índices y palabras
word_index = imdb.get_word_index()
index_word = {v+3: k for k, v in word_index.items()}
index_word[0] = "<PAD>"
index_word[1] = "<START>"
index_word[2] = "<UNK>"
index_word[3] = "<UNUSED>"

# Función para calcular sentimiento detrás de una palabra
def word_sentiment(word):
    score = analyzer.polarity_scores(word)["compound"]
    if score > 0.05:
        return 1   # Positivo
    elif score < -0.05:
        return -1  # Negativo
    return 0      # Neutran

# Función para extraer features
def extract_features(encoded_review):
    words = [index_word.get(i, "<UNK>") for i in encoded_review]
    sentiments = [word_sentiment(w) for w in words if w not in ("<PAD>", "<START>", "<UNK>", "<UNUSED>")]

    if len(sentiments) == 0:
        return 0.0, 0.0, 0.0

    pos_count = sum(1 for s in sentiments if s > 0)
    neg_count = sum(1 for s in sentiments if s < 0)
    total_count = len(sentiments)

    pos_ratio = pos_count / total_count
    neg_ratio = neg_count / total_count
    neutral_ratio = 1.0 - pos_ratio - neg_ratio

    return pos_ratio, neg_ratio, neutral_ratio

# Extraer para conjunto de entrenamiento
train_features = np.array([extract_features(review) for i, review in enumerate(x_train)])

# Extraer para conjunto de prueba
test_features  = np.array([extract_features(review) for i, review in enumerate(x_test)])

print("Extracción de Features Terminada")
print(train_features[:5])


Extracción de Features Terminada
[[0.0921659  0.01382488 0.89400922]
 [0.06382979 0.07446809 0.86170213]
 [0.04316547 0.05035971 0.90647482]
 [0.04770642 0.02201835 0.93027523]
 [0.02739726 0.11643836 0.85616438]]


Ahora, podemos utilizar padding para que las críticas tengan una longitud uniforme.

In [24]:
X_train = sequence.pad_sequences(x_train, maxlen = 80, dtype='float32')
X_test = sequence.pad_sequences(x_test, maxlen = 80, dtype='float32')

## 3. Configuración del Modelo

Ahora, para crear el modelo este debe ser configurado para tomar como entrada las secuencias al igual que los features adicionales. La justificación del modelo se encuentra en un PDF en este directorio

In [29]:
# Cambio de tipo de datos, por alguna razón tuvimos problemas con esto
train_features = train_features.astype('float32')
test_features = test_features.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Definición de inputs
sequence_input = Input(shape=(None,), name='sequence')
features_input = Input(shape=(3,), name='sentiment_features')

# Procesamiento de secuencias - embedding simple
embedded = Embedding(50000, 128)(sequence_input)
embedded = Dropout(0.4)(embedded)  # Dropout

# LSTM bidireccional - captura contexto en ambas direcciones
lstm_out = Bidirectional(LSTM(
    64,
    dropout=0.4,
    recurrent_dropout=0.4
))(embedded)

# Procesamiento de features - aprende combinaciones de features adicionales
feature_processed = Dense(8, activation='relu')(features_input)
feature_processed = Dropout(0.4)(feature_processed)  # Dropout

# Fusión - combina entendimiento de secuencia con features estadísticos
combined = concatenate([lstm_out, feature_processed])

# Clasificación - una sola capa densa es suficiente para binario
dense = Dense(32, activation='relu')(combined)
dense = Dropout(0.4)(dense)  # Dropout

# Output
output = Dense(1, activation='sigmoid')(dense)

# Crear modelo
modelo = Model(inputs=[sequence_input, features_input], outputs=output)

# Compilar modelo, learning rate = 0.0003
optimizer = Adam(learning_rate=0.0003)
modelo.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
modelo.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sequence            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 128) │  6,400,000 │ sequence[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sentiment_features  │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, None, 128) │          0 │ embedding_4[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 8)         │         32 │ sentiment_featur… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_4     │ (None, 128)       │     98,816 │ dropout_11[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 8)         │          0 │ dense_12[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 136)       │          0 │ bidirectional_4[… │
│ (Concatenate)       │                   │            │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 32)        │      4,384 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 32)        │          0 │ dense_13[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 1)         │         33 │ dropout_13[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,503,265 (24.81 MB)

 Trainable params: 6,503,265 (24.81 MB)

 Non-trainable params: 0 (0.00 B)

## 4. Entrenamiento y Prueba
Para el entrenamiento del modelo, utilizamos algunos callbacks incluyendo EarlyStopping y ReduceLROnPlateau para ayudar al modelo a converger de manera correcta.

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Problemas de tipado
x_train = pad_sequences(x_train, maxlen=80, dtype='int32')
x_test = pad_sequences(x_test, maxlen=80, dtype='int32')
train_features = np.array(train_features, dtype='float32')
test_features = np.array(test_features, dtype='float32')
y_train = np.array(y_train, dtype='float32')
y_test = np.array(y_test, dtype='float32')


# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=2,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.75,
        patience=1,
        min_lr=1e-6,
        verbose=1
    )
]

# Entrenamiento
historia = modelo.fit(
    [x_train, train_features],
    y_train,
    validation_data=([x_test, test_features], y_test),
    epochs=8,
    batch_size=128,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/8
196/196 ━━━━━━━━━━━━━━━━━━━━ 116s 562ms/step - accuracy: 0.5357 - loss: 0.6870 - val_accuracy: 0.7869 - val_loss: 0.5545 - learning_rate: 3.0000e-04
Epoch 2/8
196/196 ━━━━━━━━━━━━━━━━━━━━ 109s 558ms/step - accuracy: 0.7843 - loss: 0.4962 - val_accuracy: 0.8330 - val_loss: 0.3896 - learning_rate: 3.0000e-04
Epoch 3/8
196/196 ━━━━━━━━━━━━━━━━━━━━ 109s 556ms/step - accuracy: 0.8597 - loss: 0.3544 - val_accuracy: 0.8405 - val_loss: 0.3608 - learning_rate: 3.0000e-04
Epoch 4/8
196/196 ━━━━━━━━━━━━━━━━━━━━ 146s 575ms/step - accuracy: 0.8964 - loss: 0.2804 - val_accuracy: 0.8406 - val_loss: 0.3582 - learning_rate: 3.0000e-04
Epoch 5/8
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.9154 - loss: 0.2384
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00022500001068692654.
196/196 ━━━━━━━━━━━━━━━━━━━━ 138s 557ms/step - accuracy: 0.9154 - loss: 0.2384 - val_accuracy: 0.8344 - val_loss: 0.3716 - learning_rate: 3.0000e-04
Epoch 6/8
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/ste

## Evaluación

In [31]:
perdida, exactitud = modelo.evaluate([x_test, test_features], y_test,
                            batch_size = 64,
                            verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)

391/391 - 33s - 85ms/step - accuracy: 0.8406 - loss: 0.3582
Pérdida de la Prueba: 0.35815152525901794
Exactitud de la Prueba (Test accuracy): 0.8406400084495544
